In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification
import os
model_id='Qwen/Qwen3-0.6B'
hf_token=
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    token=hf_token)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=True
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=5,
    device_map='auto',
    quantization_config=bnb_config,
    token=hf_token  # 토큰 추가
)

model.config.use_cache = False
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
import numpy as np

# -------------------------------
# 1. 엑셀 데이터 로드
# -------------------------------

# 엑셀 파일 경로 (수정 필요)
train_excel_path = "C:/Users/nice.DESKTOP-0JCR5PF/Desktop/train.xlsx"  # 200행
test_excel_path = "C:/Users/nice.DESKTOP-0JCR5PF/Desktop/test.xlsx"    # 50행

# 엑셀 읽기
train_df = pd.read_excel(train_excel_path)
test_df = pd.read_excel(test_excel_path)

print(f"Train 데이터 크기: {train_df.shape}")
print(f"Test 데이터 크기: {test_df.shape}")
print(f"칼럼: {list(train_df.columns)}")

# -------------------------------
# 2. 텍스트 칼럼들을 합치기
# -------------------------------

text_columns = ['발명의 명칭', '요약', '전체청구항', '대표청구항']

def combine_text_columns(row):
    """4개 텍스트 칼럼을 하나로 합치기"""
    texts = []
    for col in text_columns:
        if pd.notna(row[col]) and str(row[col]).strip():  # null이 아니고 빈 문자열이 아닌 경우
            texts.append(f"{col}: {str(row[col]).strip()}")
    return " | ".join(texts)

# 텍스트 칼럼 합치기
train_df['text'] = train_df.apply(combine_text_columns, axis=1)
test_df['text'] = test_df.apply(combine_text_columns, axis=1)

print(f"\n합쳐진 텍스트 예시:")
print(train_df['text'].iloc[0][:200] + "...")

# -------------------------------
# 3. 라벨 인코딩
# -------------------------------

# 사용자태그를 라벨로 변환
label_encoder = LabelEncoder()

# train + test 전체 라벨로 인코더 학습 (일관성 유지)
all_labels = pd.concat([train_df['사용자태그'], test_df['사용자태그']])
label_encoder.fit(all_labels.dropna())

# 라벨 인코딩 적용
train_df['label'] = label_encoder.transform(train_df['사용자태그'])
test_df['label'] = label_encoder.transform(test_df['사용자태그'])

# 라벨 정보 출력
unique_labels = label_encoder.classes_
num_labels = len(unique_labels)
print(f"\n라벨 정보:")
print(f"총 라벨 수: {num_labels}")
for i, label in enumerate(unique_labels):
    print(f"  {i}: {label}")

print(f"\nTrain 라벨 분포:")
print(train_df['사용자태그'].value_counts())
print(f"\nTest 라벨 분포:")
print(test_df['사용자태그'].value_counts())

# -------------------------------
# 4. Dataset 객체로 변환
# -------------------------------

# 필요한 칼럼만 선택 (text, label)
train_dataset_dict = {
    'text': train_df['text'].tolist(),
    'label': train_df['label'].tolist()
}

test_dataset_dict = {
    'text': test_df['text'].tolist(),
    'label': test_df['label'].tolist()
}

# Dataset 객체 생성
train_data = Dataset.from_dict(train_dataset_dict)
test_data = Dataset.from_dict(test_dataset_dict)

# DatasetDict 생성
dataset = DatasetDict({
    'train': train_data,
    'test': test_data
})

print(f"\nDataset 생성 완료:")
print(f"Train: {len(dataset['train'])}개")
print(f"Test: {len(dataset['test'])}개")


In [ ]:
def preprocess_function(examples):
    tokenized = tokenizer(examples['text'], truncation=True, max_length=512)
    tokenized['labels'] = examples['label']
    return tokenized

tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
tokenized_train = tokenized_train.remove_columns(['text', 'label'])
tokenized_test = tokenized_test.remove_columns(['text', 'label'])

print(f"\n토크나이징 완료:")
print(tokenized_train[0])

In [ ]:
from transformers import DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import classification_report
import torch.nn.functional as F
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
def compute_metrics(pred):
    labels=pred.label_ids
    preds=pred.predictions.argmax(-1)
    precision, recall, f1,_ =precision_recall_fscore_support(labels, preds, average='macro')
    acc=accuracy_score(labels,preds)
    print("\n Classification Report")
    print(classification_report(labels, preds, digits=2))
    logits_tensor=torch.tensor(pred.predictions)
    labels_tensor=torch.tensor(pred.label_ids)
    loss=F.cross_entropy(logits_tensor,labels_tensor).item()
    return{
        'accuracy':acc,
        'f1':f1,
        'precision':precision,
        'recall':recall,
        'eval_loss':loss
    }

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
peft_config=LoraConfig(
    lora_alpha=128,
    lora_dropout=0.1,
    r=64,
    bias='none',
    task_type='SEQ_CLS',
    target_modules=['k_proj','gate_proj','v_proj','up_proj','q_proj','o_proj','down_proj']
)
model=prepare_model_for_kbit_training(model)
model=get_peft_model(model,peft_config)

In [ ]:
from trl import SFTConfig
output_dir= "/output"
training_arguments=SFTConfig(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    optim='paged_adamw_32bit',
    lr_scheduler_type='cosine',
    num_train_epochs=5,
    warmup_steps=50,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    dataset_text_field='text',
    max_length=512,
    label_names=['labels']
)

In [ ]:
import os
from trl import SFTTrainer
trainer=SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    processing_class=tokenizer,
    args=training_arguments,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    peft_config=peft_config
)
trainer.train()
print(trainer.evaluate())
trainer.model.save_pretrained('Qwen3-0.6B-QLoRA2')

In [ ]:
from transformers import AutoModelForSequenceClassification
from peft import PeftModel

# -------------------------------
# QLoRA SEQ_CLS 어댑터 머지
# -------------------------------

print("=== QLoRA Adapter Merge ===")

# 1. 베이스 모델을 SEQ_CLS로 직접 로드 (어댑터 훈련 시와 동일)
base_model = AutoModelForSequenceClassification.from_pretrained(
    "Qwen/Qwen3-0.6B",
    num_labels=5,
    ignore_mismatched_sizes=False
)

print("Base model structure:")
for name, _ in base_model.named_modules():
    if 'score' in name or 'classifier' in name:
        print(f"  Found: {name}")

# 2. 어댑터 로드 및 머지
adapter_path = "/Qwen3-0.6B-QLoRA"

model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    ignore_mismatched_sizes=False,
    device_map='auto'
)

# 3. 머지 및 저장
merged_model = model.merge_and_unload()
merged_model.save_pretrained("C:/wips/output/Qwen3_merged_method2")
print("머지 완료!")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from datasets import load_from_disk, DatasetDict
import numpy as np

# 데이터 로드 (기존 코드 그대로)
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
import numpy as np

# -------------------------------
# 1. 엑셀 데이터 로드 및 전처리
# -------------------------------

# 엑셀 파일 경로
train_excel_path = "C:/Users/nice.DESKTOP-0JCR5PF/Desktop/train.xlsx"  # 200행
test_excel_path = "C:/Users/nice.DESKTOP-0JCR5PF/Desktop/test.xlsx"    # 50행

# 엑셀 읽기
train_df = pd.read_excel(train_excel_path)
test_df = pd.read_excel(test_excel_path)

# 텍스트 칼럼들을 합치기
text_columns = ['발명의 명칭', '요약', '전체청구항', '대표청구항']

def combine_text_columns(row):
    texts = []
    for col in text_columns:
        if pd.notna(row[col]) and str(row[col]).strip():
            texts.append(f"{col}: {str(row[col]).strip()}")
    return " | ".join(texts)

train_df['text'] = train_df.apply(combine_text_columns, axis=1)
test_df['text'] = test_df.apply(combine_text_columns, axis=1)

# 라벨 인코딩
label_encoder = LabelEncoder()
all_labels = pd.concat([train_df['사용자태그'], test_df['사용자태그']])
label_encoder.fit(all_labels.dropna())

train_df['label'] = label_encoder.transform(train_df['사용자태그'])
test_df['label'] = label_encoder.transform(test_df['사용자태그'])

# Dataset 객체로 변환
train_data = Dataset.from_dict({
    'text': train_df['text'].tolist(),
    'label': train_df['label'].tolist()
})

test_data = Dataset.from_dict({
    'text': test_df['text'].tolist(),
    'label': test_df['label'].tolist()
})

# -------------------------------
# 2. 기존 방식과 동일한 샘플링
# -------------------------------

# 기존 코드와 동일한 구조
ds = DatasetDict({'train': train_data, 'test': test_data})
train = ds["train"]
test = ds["test"]

num_labels = len(set(train["label"]))
train_per_label = 200 // num_labels  # 전체 200개를 라벨별로 균등 분배
test_per_label = 50 // num_labels    # 전체 50개를 라벨별로 균등 분배

print(f"라벨 수: {num_labels}")
print(f"라벨별 train 샘플: {train_per_label}개")
print(f"라벨별 test 샘플: {test_per_label}개")

def sample_per_label(dataset, per_label, seed=42):
    labels = np.array(dataset["label"])
    picked = []
    rng = np.random.default_rng(seed)
    for l in range(num_labels):
        idx = np.where(labels == l)[0]
        if len(idx) < per_label:
            print(f"경고: 라벨 {l}에 {len(idx)}개 샘플만 있음 (필요: {per_label}개)")
            picked.extend(idx)  # 있는 만큼만 추가
        else:
            picked.extend(rng.choice(idx, per_label, replace=False))
    return dataset.select(sorted(picked))

train_small = sample_per_label(train, train_per_label)
test_small = sample_per_label(test, test_per_label)

# 전체 데이터 (실제로는 train 200 + test 50 = 250개)
all_texts = list(train_small["text"]) + list(test_small["text"])
all_labels = list(train_small["label"]) + list(test_small["label"])

print(f"\n전체 샘플: {len(all_texts)}개")
print(f"Train 샘플: {len(train_small)}개")
print(f"Test 샘플: {len(test_small)}개")

# 라벨 분포 확인
from collections import Counter
print(f"\nTrain 라벨 분포: {Counter(train_small['label'])}")
print(f"Test 라벨 분포: {Counter(test_small['label'])}")

# 원본 라벨로 변환해서 확인
train_original_labels = [label_encoder.inverse_transform([label])[0] for label in train_small['label']]
test_original_labels = [label_encoder.inverse_transform([label])[0] for label in test_small['label']]

print(f"\nTrain 원본 라벨 분포: {Counter(train_original_labels)}")
print(f"Test 원본 라벨 분포: {Counter(test_original_labels)}")

# 모델 로드 (GPU 강제 사용)
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"사용할 디바이스: cuda:0")

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델 로드 (device_map 사용하지 않고 직접 이동)
model = AutoModelForSequenceClassification.from_pretrained(
    "/output/Qwen3_merged_method2",
    num_labels=5,
    torch_dtype=torch.float16  # GPU 메모리 절약
)

# GPU로 강제 이동
device = torch.device("cuda:0")
model = model.to(device)
model.config.pad_token_id = tokenizer.pad_token_id

print(f"모델 디바이스: {next(model.parameters()).device}")
print(f"모델 dtype: {next(model.parameters()).dtype}")

# GPU 메모리 확인
torch.cuda.empty_cache()
print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

# 예측
model.eval()
predictions = []

with torch.no_grad():
    for i in range(0, len(all_texts), 8):  # 배치 8
        batch_texts = all_texts[i:i+8]
        inputs = tokenizer(batch_texts, truncation=True, padding=True,
                          max_length=512, return_tensors="pt").to(device)

        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# 결과
accuracy = accuracy_score(all_labels, predictions)

# 학습 데이터 vs 검증 데이터 비교
train_acc = accuracy_score(train_small["label"], predictions[:200])
val_acc = accuracy_score(test_small["label"], predictions[200:])
